In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
import time
import shutil
import os
from datetime import datetime
currentDateTime = datetime.now()
print("Date of Today : ", currentDateTime.month, " /", currentDateTime.day, "\nHour : ", currentDateTime.hour) 

#Date of Today
ctime = f"{currentDateTime.month}_{currentDateTime.day}_{currentDateTime.hour}h"
#torch.autograd.detect_anomaly(check_nan=True)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

#Choose Test Problem
from BM_WZ_CHnet import BM_WZ_CHnet
Net = BM_WZ_CHnet
#torch.autograd.set_detect_anomaly(True)

#Create and Train the network
def create_network(layers, preload = False, preload_name=''):
    
    start = time.time()
    
    
    net = Net(layers)
    net = net.to(device)
    
    if preload == True:
        try:
            net.load_state_dict(torch.load(preload_name, map_location=torch.device('cpu')))
            print('Loaded Successfully')
        except Exception as error:
            print('Loading Failed: ', error)
            pass
        
    time_vec = [0, 0, 0, 0]
    
    #Set final times for running training_loop
    time_slices = np.array([1])
    
    global epsilon #used to track loss
    epsilon = []
    global a_u_set
    a_u_set = []
    global a_u_diff_set
    a_u_diff_set = []
    
    global b_u_set
    b_u_set = []
    global c_ac_set
    c_ac_set = []
    
    a_u = 1 ##10 
    a_u_diff = 1
    
    b_u = 1000 ##100 
    c_ac = 1 ###1/10 ##1000
    
    
    print('Training PDE')
    
    iteration_vec = [120000]
    learning_rate_vec = 1e-3 * np.ones_like(iteration_vec)
    #r_vec = [30, 20, 10, 10]
    
    
    for i in range(len(iteration_vec)):
        #Set loop to optimize in progressively smaller learning rates
        print(f'Executing Pass {i+1}')
        iterations = iteration_vec[i]
        learning_rate = learning_rate_vec[i]   
        #r = r_vec[i] #determines sampling distribution decay rate
        r=1 
        
        training_loop(net, time_slices, iterations, learning_rate, r, record_loss = 100, print_loss = 500, a_u = a_u, a_u_diff = a_u_diff, b_u = b_u, c_ac = c_ac )
        torch.save(net.state_dict(), f"CH_Benchmarks_Pass_{i+1}.pt")
        
        time_slices = [time_slices[-1]]
        
        np.savetxt('epsilon.txt', epsilon)
        time_vec[i] = time.time()


    np.savetxt('epsilon.txt', epsilon)
    
    end = time.time()

    print("Total Time:\t", end-start)
    for i in range(len(iteration_vec)):
        if i>0:
            print(f'Pass {i+1} Time: {time_vec[i] - time_vec[i-1]}')
        else:
            print(f'Pass 1 Time: {time_vec[0] - start}')


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def exponential_time_sample(collocation, t_l, t_u, r):
    t_pre = np.random.uniform(0,1, size=(collocation, 1))
    
    t = -np.log(1-t_pre+t_pre*np.exp(-r*(t_u-t_l)))/r
    
    return t

def initialize_adaptive_x(net, x_l, x_u, size_IC, size_domain):
    total_size = size_IC + size_domain
    #Initialize
    x = np.random.uniform(low=x_l, high=x_u, size=(total_size,1))
    #y = np.random.uniform(low=y_l, high=y_u, size=(total_size,1))
    
    its = 10000
    for i in range(its):
        x_pt = Variable(torch.from_numpy(x).float(), requires_grad=True).to(device)
        #y_pt = Variable(torch.from_numpy(y).float(), requires_grad=True).to(device)
        
        psi = net.W_exact(x_pt) #calculate density (scaled by constant)
        psi = psi.detach().cpu().numpy()
        
        #First Proposal
        x_prop = np.random.uniform(low=x_l, high=x_u, size=(total_size,1))
        #y_prop = np.random.uniform(low=y_l, high=y_u, size=(total_size,1))
        
        x_prop_pt = Variable(torch.from_numpy(x_prop).float(), requires_grad=True).to(device)
        #y_prop_pt = Variable(torch.from_numpy(y_prop).float(), requires_grad=True).to(device)
        psi_prop = net.W_exact(x_prop_pt) #calculate density (scaled by constant)
        psi_prop = psi_prop.detach().cpu().numpy()
        
        a = psi_prop/psi
        
        #accept worse proposals with probability a
        rand = np.random.uniform(low=0, high=1, size=(total_size,1))
        x[rand<a] = x_prop[rand<a]
        #y[rand<a] = y_prop[rand<a]

    x_IC, x_dom = np.split(x, [size_IC])
    #y_IC, y_dom = np.split(y, [size_IC])
    return x_IC, x_dom

def step_adaptive_x(net, x_IC, x_dom, t_dom, x_l, x_u):
    x_IC = x_IC.detach()
    #y_IC = y_IC.detach()
    x_dom = x_dom.detach() 
    #y_dom = y_dom.detach()
    t_dom = t_dom.detach()
    
    Psi_IC = net.W_exact(x_IC)
    Psi_dom = net.W(x_dom, t_dom)
    
    its = 1
    for i in range(its):    
        #First Proposal
        x_prop_IC = (x_u-x_l)*torch.rand_like(x_IC) + x_l
        #y_prop_IC = (y_u-y_l)*torch.rand_like(y_IC) + y_l
        
        x_prop_dom = (x_u-x_l)*torch.rand_like(x_dom) + x_l
        #y_prop_dom = (y_u-y_l)*torch.rand_like(y_dom) + y_l
        
        psi_prop_IC = net.W_exact(x_prop_IC) #calculate density (scaled by constant)
        psi_prop_dom = net.W(x_prop_dom, t_dom)
        
        a_IC = psi_prop_IC/Psi_IC
        a_dom = psi_prop_dom/Psi_dom
        
        #accept worse proposals with probability a
        rand_IC = torch.rand_like(x_IC)
        rand_dom = torch.rand_like(x_dom)
        
        x_IC[rand_IC<a_IC] = x_prop_IC[rand_IC<a_IC]
        #y_IC[rand_IC<a_IC] = y_prop_IC[rand_IC<a_IC]
        
        x_dom[rand_dom<a_dom] = x_prop_dom[rand_dom<a_dom]
        #y_dom[rand_dom<a_dom] = y_prop_dom[rand_dom<a_dom]

    return x_IC, x_dom

def autograd_W(net, input):
    list_grad = []
    layers = [[net.layers_u[0].weight,net.layers_u[1].weight,net.layers_u[2].weight,net.layers_u[3].weight, net.layers_u[4].weight,net.layers_u[5].weight,net.layers_u[6].weight]]
    #[net.hidden_layer1_u1.weight,net.hidden_layer2_u1.weight,net.hidden_layer3_u1.weight, net.hidden_layer4_u1.weight,net.output_layer_u1.weight],
                #[net.hidden_layer1_u2.weight,net.hidden_layer2_u2.weight,net.hidden_layer3_u2.weight, net.hidden_layer4_u2.weight,net.output_layer_u2.weight], 
                 #[net.hidden_layer1_P.weight,net.hidden_layer2_P.weight,net.hidden_layer3_P.weight, net.hidden_layer4_P.weight,net.output_layer_P.weight],
                  
    for j in range(0, len(layers)):
        for i in range(0,7):
            diff = layers[j][i]
            try:
                result = torch.autograd.grad(input.sum(), diff, create_graph=True, retain_graph=True)[0]
                list_grad.append(result)
            except:
                pass
    return list_grad

def training_loop(net, time_slices, iterations, learning_rate, r, record_loss, print_loss, a_u, a_u_diff, b_u, c_ac):
    global epsilon #used to track and record loss
    global a_u_set
    global a_u_diff_set
    global b_u_set
    global c_ac_set
    
    # Domain boundary
    x_l = net.x1_l
    x_u = net.x1_u
    
    #y_l = net.x2_l
    #y_u = net.x2_u

    #time starts at lower bound 0, ends at upper bouund updated in slices
    t_l = 0

    # Generate following random numbers for x, y t
    BC_collocation = int(160) #1000
    IC_collocation = int(100) #5000
    pde_collocation = int(2000) #10000
    
    IC_collocation_adaptive = int(40) #5000
    pde_collocation_adaptive = int(8000) #10000
    lamba = .9
    #learning rate update
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    for final_time in time_slices:
        min_loss = 20
        min_mse_BC = .01
        
        min_mse_AC = .01
        min_mse_IC = .01
        
        epoch = 0
         
        with torch.autograd.no_grad():
            print("Current Final Time:", final_time, "Current Learning Rate: ", get_lr(net.optimizer))  
        
        #Initialize Adaptive Sampling points outside the loop
        ##Define Colloacation Points with Initial Condition
        
        x_IC_adaptive, x_domain_adaptive = initialize_adaptive_x(net, x_l, x_u, IC_collocation_adaptive, pde_collocation_adaptive)

        input_x_IC_adaptive = Variable(torch.from_numpy(x_IC_adaptive).float(), requires_grad=True).to(device)
        #input_y_IC_adaptive = Variable(torch.from_numpy(y_IC_adaptive).float(), requires_grad=True).to(device)

        #Define Inner Domain Collocation Points
        t_domain_adaptive = exponential_time_sample(pde_collocation_adaptive, t_l, final_time, r)

        input_x_domain_adaptive = Variable(torch.from_numpy(x_domain_adaptive).float(), requires_grad=True).to(device)
        #input_y_domain_adaptive = Variable(torch.from_numpy(y_domain_adaptive).float(), requires_grad=True).to(device)
        input_t_domain_adaptive = Variable(torch.from_numpy(t_domain_adaptive).float(), requires_grad=True).to(device)
        
        #grad_NS_loss_mean = 0.1 **5
        #grad_NS_div_loss_mean = 0.1 **5
        grad_AC_loss_mean = 1
        
        grad_BC_u_loss_mean = 1
        grad_BC_u_diff_loss_mean = 1
        grad_IC_u_loss_mean = 1000
        
        
        epoch = 0
        while epoch <= iterations:
            # Resetting gradients to zero            
            net.optimizer.zero_grad()    
            
            if epoch > 0:
                input_x_IC_adaptive, input_x_domain_adaptive = step_adaptive_x(net, input_x_IC_adaptive, input_x_domain_adaptive, input_t_domain_adaptive, x_l, x_u)
                
                input_x_IC_adaptive = Variable(input_x_IC_adaptive, requires_grad=True).to(device)
                #input_y_IC_adaptive = Variable(input_y_IC_adaptive, requires_grad=True).to(device)
                
                input_x_domain_adaptive = Variable(input_x_domain_adaptive, requires_grad=True).to(device)
                #input_y_domain_adaptive = Variable(input_y_domain_adaptive, requires_grad=True).to(device)
                input_t_domain_adaptive = Variable(input_t_domain_adaptive, requires_grad=True).to(device)
                
            ##Define Colloacation Points with Initial Condition
            x_IC = np.random.uniform(low=x_l, high=x_u, size=(IC_collocation,1))
            #y_IC = np.random.uniform(low=y_l, high=y_u, size=(IC_collocation,1))

            input_x_IC = Variable(torch.from_numpy(x_IC).float(), requires_grad=True).to(device)
            #input_y_IC = Variable(torch.from_numpy(y_IC).float(), requires_grad=True).to(device)
            
            ##Define Boundary Condition Collocation Points
            x_BC = np.random.uniform(low=x_l, high=x_u, size=(BC_collocation,1))
            #y_BC = np.random.uniform(low=y_l, high=y_u, size=(BC_collocation,1))       
            t_BC = np.random.uniform(low=t_l, high=final_time, size=(BC_collocation,1))
    
            input_x_BC= Variable(torch.from_numpy(x_BC).float(), requires_grad=True).to(device)
            #input_y_BC = Variable(torch.from_numpy(y_BC).float(), requires_grad=True).to(device)
            input_t_BC = Variable(torch.from_numpy(t_BC).float(), requires_grad=True).to(device)
    
            #Define Inner Domain Collocation Points
            x_domain = np.random.uniform(low= x_l, high=x_u, size=(pde_collocation, 1))
            #y_domain = np.random.uniform(low= y_l, high=y_u, size=(pde_collocation, 1))
            t_domain = exponential_time_sample(pde_collocation, t_l, final_time, r)
            #t_domain = np.random.uniform(low= t_l, high=final_time, size=(pde_collocation, 1)) 
    
            input_x_domain = Variable(torch.from_numpy(x_domain).float(), requires_grad=True).to(device)
            #input_y_domain = Variable(torch.from_numpy(y_domain).float(), requires_grad=True).to(device)
            input_t_domain = Variable(torch.from_numpy(t_domain).float(), requires_grad=True).to(device)
            
            ### Training steps
            ## Compute Loss
            # Loss based on Initial Condition
            mse_IC = net.Initial_Condition_Loss(input_x_IC)
            mse_IC_adaptive = net.Initial_Condition_Loss(input_x_IC_adaptive)
            
            mse_IC_total = mse_IC + mse_IC_adaptive
        
            # Loss based on Boundary Condition (Containing No-Slip and Free-slip)
            mse_BC_u, mse_BC_u_diff = net.Boundary_Loss(input_x_BC, input_t_BC)
        
            # Loss based on PDE
            mse_PDE = net.PDE_Loss(input_x_domain, input_t_domain)
            mse_PDE_adaptive = net.PDE_Loss(input_x_domain_adaptive, input_t_domain_adaptive)
            
            mse_PDE_total = mse_PDE #+ mse_PDE_adaptive
            
            # Sum Loss
            loss = b_u * mse_IC_total + a_u *mse_BC_u+ a_u_diff * mse_BC_u_diff + c_ac * mse_PDE_total
            
            '''
            if epoch%1 == 0:
                
                #compute next step
                
                if c_ac < 5000 :
                    max_set = []
                    for e in autograd_W(net, mse_PDE_total):
                        m = torch.max(torch.abs(e))
                        max_set.append(m.item())
                    grad_AC_loss_mean = np.max(max_set)
                
                if a_u < 5000 :
                    mean_set = []
                    for e in autograd_W(net, mse_BC_u):
                        m = torch.mean(torch.abs(e))
                        mean_set.append(m.item())
                    grad_BC_u_loss_mean = np.mean(mean_set)
                
                if a_u < 5000 :
                    mean_set = []
                    for e in autograd_W(net, mse_BC_u_diff):
                        m = torch.mean(torch.abs(e))
                        mean_set.append(m.item())
                    grad_BC_u_diff_loss_mean = np.mean(mean_set)
                
                if b_u < 5000 :
                    mean_set = []
                    for e in autograd_W(net, mse_IC_total):
                        m = torch.mean(torch.abs(e))
                        mean_set.append(m.item())
                    grad_IC_u_loss_mean = np.mean(mean_set)
                
                #grad_mean_sum = (grad_BC_u_loss_mean + grad_BC_u_diff_loss_mean + grad_IC_u_loss_mean+ grad_AC_loss_mean )
                
                
                # coeffiecient of BC => a, coeffiecient of IC => b
                a_u_next = (grad_AC_loss_mean)/(grad_BC_u_loss_mean)
                a_u_diff_next = (grad_AC_loss_mean)/(grad_BC_u_diff_loss_mean)
                
                b_u_next = (grad_AC_loss_mean)/(grad_IC_u_loss_mean)
                
                #c_ac_next = (grad_mean_sum)/(grad_AC_loss_mean)
                
                # #update rates
                if a_u_next < 5000:
                    a_u = (1-lamba) * a_u + lamba * a_u_next
                if a_u_diff_next < 5000:
                    a_u_diff = (1-lamba) * a_u_diff + lamba * a_u_diff_next
                
                if b_u_next < 5000:
                    b_u = (1-lamba) * b_u + lamba * b_u_next
                #if c_ac_next < 5000:
                    #c_ac = (1-lamba) * c_ac + lamba * c_ac_next
            '''
            
            ## Compute Gradients
            # Compute Actual Gradients
            loss.backward()
            
            #Gradient Norm Clipping
            #torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm = 1000, error_if_nonfinite=False) #note max norm is more art than science
            
            def closure():
                return loss
            
            ## Step Optimizer
            net.optimizer.step(closure)
            #net.optimizer.step()
            
            ### Update Epoch
            epoch = epoch + 1
            
            ### Save and Print
            
            # if epoch < 10:
            #     #save adaptive points
            #     torch.save(input_x_IC_adaptive, f'x_IC_adaptive_{epoch}.pt')
            #     torch.save(input_y_IC_adaptive, f'y_IC_adaptive_{epoch}.pt')
            #     torch.save(input_x_domain_adaptive, f'x_dom_adaptive_{epoch}.pt')
            #     torch.save(input_y_domain_adaptive, f'y_dom_adaptive_{epoch}.pt')
            #     torch.save(input_t_domain_adaptive, f't_dom_adaptive_{epoch}.pt')
            #Print Loss every 1000 Epochs
            with torch.autograd.no_grad():
                if epoch == 1 or epoch%record_loss == 0:
                    epsilon = np.append(epsilon, loss.cpu().detach().numpy())
                    #a_u_set = np.append(a_u_set, a_u)
                    #a_u_diff_set = np.append(a_u_diff_set, a_u_diff)
                    #b_u_set = np.append(b_u_set, b_u)
                    #c_ac_set = np.append(c_ac_set, c_ac)
                if epoch == 1 or epoch%print_loss == 0:
                    print("\nIteration:", epoch, "\tTotal Loss:", loss.data)
                    print("\tphi IC Loss: ", mse_IC.item(),"\tphi IC Loss Adaptive: ", mse_IC_adaptive.item(),"\tu BC Loss: ", mse_BC_u.item(), 
                          "\tu diff BC Loss: ", mse_BC_u_diff.item(), "\nPDE Loss: ", mse_PDE.item(), "\tPDE Loss: ", mse_PDE_adaptive.item())
                    
                    if mse_PDE_total.cpu().detach().numpy() < min_mse_AC:
                        min_mse_AC = mse_PDE_total.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_CHloss_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the min CH PDE Loss\n')
                    if mse_BC_u.cpu().detach().numpy() < min_mse_BC:
                        min_mse_BC = mse_BC_u.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_mse_Bd_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the min Bd Loss\n')
                    if mse_IC_total.cpu().detach().numpy() < min_mse_IC:
                        min_mse_IC = mse_IC_total.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_mse_IC_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the min IC Loss\n')
                    if loss.cpu().detach().numpy() < min_loss:
                        min_loss = loss.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_loss_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the Minimal Total Loss\n')
                if epoch%6000 == 0:
                    np.savetxt(f"{ctime}epsilon.txt", epsilon)
                    #np.savetxt(f"{ctime}_a_u.txt", a_u_set)
                    #np.savetxt(f"{ctime}_a_u_diff.txt", a_u_diff_set)
                    #np.savetxt(f"{ctime}_b_u.txt", b_u_set)
                    #np.savetxt(f"{ctime}_c_ac.txt", c_ac_set)
                    torch.save(net.state_dict(), f"lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                    
    
layers=[2, 128, 128, 128, 128, 128, 128, 1]
create_network(layers, preload=True, preload_name='lr0.001_t0.2_3_4_15h.pt')


Date of Today :  3  / 6 
Hour :  10
Loaded Successfully
Training PDE
Executing Pass 1
Current Final Time: 1 Current Learning Rate:  0.001


/var/folders/j9/4s8713nn53v3jz3ky1rz1f6h0000gn/T/ipykernel_2098/830190865.py:133: RuntimeWarning: divide by zero encountered in divide
  a = psi_prop/psi



Iteration: 1 	Total Loss: tensor(2.1281)
	phi IC Loss:  1.759724455041578e-06 	phi IC Loss Adaptive:  3.24731740874995e-06 	u BC Loss:  0.002944197040051222 	u diff BC Loss:  0.008284034207463264 
PDE Loss:  2.111903429031372 	PDE Loss:  5.063554286956787
  *Saved ; Early Stopping for the min Bd Loss

  *Saved ; Early Stopping for the min IC Loss

  *Saved ; Early Stopping for the Minimal Total Loss


Iteration: 500 	Total Loss: tensor(0.3020)
	phi IC Loss:  2.0627032427000813e-05 	phi IC Loss Adaptive:  2.3899545340100303e-05 	u BC Loss:  0.0002260564360767603 	u diff BC Loss:  0.0008047778974287212 
PDE Loss:  0.25643688440322876 	PDE Loss:  0.3630935847759247
  *Saved ; Early Stopping for the min Bd Loss

  *Saved ; Early Stopping for the Minimal Total Loss


Iteration: 1000 	Total Loss: tensor(0.3348)
	phi IC Loss:  1.6710406271158718e-05 	phi IC Loss Adaptive:  5.676093860529363e-05 	u BC Loss:  0.00026840242207981646 	u diff BC Loss:  0.0013995026238262653 
PDE Loss:  0.25967371